In [1]:
!pip install weaviate-client tiktoken pypdf rapidocr-onnxruntime
!pip install -U langchain-community

In [10]:
from google.colab import userdata
WEAVIATE_CLUSTER = userdata.get('WEAVIATE_CLUSTER')

In [11]:
from google.colab import userdata
WEAVIATE_API_KEY= userdata.get('WEAVIATE_API_KEY')

In [12]:
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.auth import AuthApiKey

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY

client = weaviate.connect_to_weaviate_cloud(
    cluster_url = WEAVIATE_URL,auth_credentials=AuthApiKey(WEAVIATE_API_KEY)
)

# Embeddings

In [20]:
!pip install sentence-transformers

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = 'sentence-transformers/all-mpnet-base-v2'
#model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    #model_kwargs=model_kwargs
)

/tmp/ipython-input-1059231990.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
import os
# Create a folder named 'pdfs'
os.makedirs("pdfs", exist_ok=True)
print("Folder created:", os.listdir())

Folder created: ['.config', 'pdfs', 'sample_data']


In [16]:
from google.colab import files

uploaded = files.upload()

# Move uploaded PDFs into the 'pdfs' folder
import shutil

for filename in uploaded.keys():
    shutil.move(filename, "pdfs/" + filename)

Saving RAG-6.pdf to RAG-6.pdf
Saving RAG-5.pdf to RAG-5.pdf
Saving RAG-4.pdf to RAG-4.pdf
Saving RAG-3.pdf to RAG-3.pdf


In [18]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

# load all PDFs from a folder
loader = DirectoryLoader(
    path="/content/pdfs",
    glob="*.pdf",
    loader_cls=PyPDFLoader,
)

pages = loader.load()

In [19]:
pages

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-06-06T01:09:34+00:00', 'author': '', 'keywords': '', 'moddate': '2024-06-06T01:09:34+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/pdfs/RAG-3.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Preprint, Under Review\nRAFT: Adapting Language Model to Domain Specific RAG\nTianjun Zhang ∗\nDepartment of Computer Science\nUC Berkeley\nBerkeley, CA 94720, USA\n{tianjunz}@berkeley.edu\nShishir G. Patil, Naman Jain, Sheng Shen\nDepartment of Computer Science\nUC Berkeley\nBerkeley, CA 94720, USA\n{shishirpatil,naman_jain,sheng.s}@berkeley.edu\nMatei Zaharia, Ion Stoica, Joseph E. Gonzalez\nDepartment of Computer Science\nUC Berkeley\nBerkeley, CA 94720, USA\n{matei,istoica,jegonzal}@berkeley.edu\nAbstract\nPretraining Large Language

# Chunking

In [21]:
#split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)
docs = text_splitter.split_documents(pages)

In [22]:
docs

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-06-06T01:09:34+00:00', 'author': '', 'keywords': '', 'moddate': '2024-06-06T01:09:34+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/pdfs/RAG-3.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Preprint, Under Review\nRAFT: Adapting Language Model to Domain Specific RAG\nTianjun Zhang ∗\nDepartment of Computer Science\nUC Berkeley\nBerkeley, CA 94720, USA\n{tianjunz}@berkeley.edu\nShishir G. Patil, Naman Jain, Sheng Shen\nDepartment of Computer Science\nUC Berkeley\nBerkeley, CA 94720, USA\n{shishirpatil,naman_jain,sheng.s}@berkeley.edu\nMatei Zaharia, Ion Stoica, Joseph E. Gonzalez\nDepartment of Computer Science\nUC Berkeley\nBerkeley, CA 94720, USA\n{matei,istoica,jegonzal}@berkeley.edu\nAbstract\nPretraining Large Language

In [24]:
# Create a collection
collection = client.collections.create(
    name="PDFDocuments",
    vectorizer_config=weaviate.classes.config.Configure.Vectorizer.none()
)

In [26]:
# Insert chunks manually
for i, doc in enumerate(docs):
    collection.data.insert(
        properties={"text": doc.page_content, "source": doc.metadata.get("source")},
        vector=embeddings.embed_query(doc.page_content)
    )

# similarity search

In [28]:
query = "what is RAG?"
query_vector = embeddings.embed_query(query)

results = collection.query.near_vector(
    near_vector=query_vector,
    limit=3
)

for o in results.objects:
    print("🔹", o.properties["text"][:400])
    print("-" * 80)

🔹 into the model to guide its response (Zhou et al., 2024).
A standard RAG system operates in two phases:
• Retrieval. A retriever model selects the top-k most relevant documents for a given query.
• Generation. A language model generates a response conditioned on both the query and the
retrieved documents.
The appeal of RAG lies in its ability to dynamically access up-to-date or domain-specific inf
--------------------------------------------------------------------------------
🔹 the model with imperfect RAG samples that mimic real-world retrieval scenarios. We constructed a
diverse dataset covering legal documents, scientific literature, books, and web data, each paired with
∗Work was done during an internship at Pints AI
2https://github.com/Pints-AI/Finetune-Bench-RAG
3https://huggingface.co/datasets/pints-ai/Finetune-RAG
--------------------------------------------------------------------------------
🔹 varied and robust scenarios that the model can train on.
• Compute requirements:

In [29]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrived context to answer the question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use ten sentences maximum and keep the answer concise.
{context}
Question: {question}
Helpful Answer:"""

In [30]:
prompt=ChatPromptTemplate.from_template(template)

In [31]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrived context to answer the question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nUse ten sentences maximum and keep the answer concise.\n{context}\nQuestion: {question}\nHelpful Answer:"), additional_kwargs={})])

In [32]:
from langchain import HuggingFaceHub

# Loading the model using Hugging-FACE API

In [33]:
from google.colab import userdata
huggingfacehub_api_token= userdata.get('HF_TOKEN')

In [34]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

/tmp/ipython-input-2949055110.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model = HuggingFaceHub(


# Generate response

In [35]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [36]:
# Output parser
output_parser = StrOutputParser()

# creating a custom retriever

In [60]:
from langchain.schema import Document

def v4_retriever_texts(query: str, collection, embeddings, k=3):
    # Embed the query
    query_vector = embeddings.embed_query(query)

    # Query Weaviate v4
    results = collection.query.near_vector(
        near_vector=query_vector,
        limit=k
    )

    # Return list of strings
    return [obj.properties["text"] for obj in results.objects]


In [61]:
retriever= v4_retriever_texts(query, collection, embeddings, k=3)

rag_chain =(
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>

In [59]:
query = "What is RAG-system?"
retrieved_texts = retriever._invoke(query)  # returns list of strings

# Combine context into a single string
context = "\n".join(retrieved_texts)

prompt_input = {"context": context, "question": query}

result = rag_chain.invoke(prompt_input)
print(result)

AttributeError: 'V4WeaviateRetriever' object has no attribute '_invoke'